# Eventdata und Verhältnisberechnung für FUR/WET

In [10]:
import matplotlib
matplotlib.use("Agg")  

In [11]:
import os
import gc
from functions.hilfsfunktionen import safe_divide, transform_to_rtz
import obspy as obs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import hilbert, savgol_filter
import plotly.graph_objects as go
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from pprint import pprint
from functions.add_distances_and_backazimuth import __add_distances_and_backazimuth
from functions.querrySeismoData import __querrySeismoData
from obspy import read
import glob
import sys
from scipy import fftpack
from andbro__fft import __fft
from obspy.taup import TauPyModel

### Methods

In [12]:
from obspy.taup import TauPyModel
def getStarttime(origin_time,distance_deg, ev_depth):
    model = TauPyModel(model="prem")
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg, phase_list=["P"])
    p_arrival_time = arrivals[0].time  # Zeit in Sekunden relativ zum origin_time
    starttime = origin_time + p_arrival_time - 20
    return starttime
font_title=17
font_ax=13
font_leg=11

In [13]:
#nur für translation --> nicht für rotation
def __makeplotspectraTsmooth(st, config, ev_lat, ev_lon, backazimuth):

    st_in = st.select(component="T")
    if len(st_in) == 0:
        print(" -> keine Z-Komponente gefunden.")
        return

    trans_scaling = 1e6
    trans_unit = r"$\mathrm{\mu m/s}$"
    font = 14

    # Initialisierung
    tr1, tr2 = None, None
    
    # Stationen gezielt zuordnen
    for tr in st_in:
        if tr.stats.station.upper() == "FUR":
            tr1 = tr
            data1 = tr1.data * trans_scaling
            spec1, ff1, ph1 = __fft(data1, tr.stats.delta, window=None, normalize=None)
        elif tr.stats.station.upper() == "WET":
            tr2 = tr
            data2 = tr2.data *trans_scaling
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
    
    # Sicherstellen, dass beide Stationen gefunden wurden
    if tr1 is None or tr2 is None:
        print(" -> Fehlende Station: FUR oder WET nicht im Stream.")
        return
    if ev_lat > 30 and ev_lat < 60 and ev_lon > -50 and ev_lon < 50:
        region = 1
        print("Region 1 (Nahbeben)")
    elif ev_lat > 15 and ev_lat < 60 and ev_lon > 60 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > -10 and ev_lat < 15 and ev_lon > 100 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > 5 and ev_lat < 60 and ev_lon > -170 and ev_lon < -60:
        region = 4
        print("Region 3 (Nord- und Südamerika)")
    elif ev_lat > -80 and ev_lat < 5 and ev_lon > -120 and ev_lon < -50:
        region = 4
        print("Region 3 (Nord- und Südamerika)")
    elif ev_lat > -60 and ev_lat < -10 and (ev_lon > 90 or ev_lon < -150):
        region = 6
        print("Region 4 (Ozeanien)")
    else:
        region = 7
        print("Region 5 (nicht zugeordnet)")      
    min_len = min(len(spec1), len(spec2), len(ff1), len(ff2))
    spec1 = spec1[:min_len]
    spec2 = spec2[:min_len]
    ff1 = ff1[:min_len]
    ff2 = ff2[:min_len]  
    envelope1 = np.abs(hilbert(tr1.data * trans_scaling))
    envelope2 = np.abs(hilbert(tr2.data * trans_scaling))
    spec1_s = pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec2_s = pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec1_s = savgol_filter(spec1_s, window_length=81, polyorder=3)
    spec2_s = savgol_filter(spec2_s, window_length=81, polyorder=3)
    lines = ["⌀ Spectral ratio:"]
    
    ratio = safe_divide(spec1_s, spec2_s)
    freq_bands = [(round(f, 5), round(f + 0.1, 5)) for f in np.arange(0.01, 1.9 + 0.001, 0.05)]

    # Mittelwerte und Mittenfrequenzen extrahieren
    band_centers = [(fmin + fmax)/2 for fmin, fmax in freq_bands]
    mean_ratios = []
    if not isinstance(ratio, int):
        for fmin_r, fmax_r in freq_bands:
            mask = (ff1 >= fmin_r) & (ff1 <= fmax_r)
            if np.any(mask):
                mean_ratios.append(np.mean(ratio[mask]))
            else:
                mean_ratios.append(np.nan)

     # Farben definieren
    colors = {
        'tr1': '#1f77b4',        # Blau
        'envelope1': '#2ca02c',  # Grün
        'tr2': '#ff7f0e',        # Orange
        'envelope2': '#17becf',  # Türkis
        'spec1': '#1f77b4',      # Blau
        'spec2': '#ff7f0e',      # Orange
    }   
    unit = trans_unit
     
    fig, axes = plt.subplots(5,figsize=(14,4*3), sharex=False)
    plt.subplots_adjust(hspace=0.15) 
    
    bbox0 = axes[0].get_position()
    bbox1 = axes[1].get_position()
    axes[1].set_position([bbox1.x0, bbox0.y0 - bbox0.height, bbox1.width, bbox1.height])
    bbox2 = axes[2].get_position()
    bbox3 = axes[3].get_position()
    axes[3].set_position([bbox3.x0, bbox2.y0 - bbox2.height, bbox3.width, bbox3.height])
    bbox3 = axes[3].get_position()
    bbox4 = axes[4].get_position()
    axes[4].set_position([bbox4.x0, bbox3.y0 - bbox3.height, bbox4.width, bbox4.height])

    axes[0].plot(tr1.times(), data1, label=f"{tr1.stats.station} {tr1.stats.channel}", color=colors['tr1'], lw=1.0)
    axes[0].plot(tr1.times(), envelope1, color=colors['envelope1'], label='Envelope ' + tr1.stats.channel) 
    axes[0].set_ylabel(f'v ({unit})', fontsize=font_ax)
    axes[0].legend(fontsize=font_leg)
    axes[0].text(0.02, 0.95, "a)", transform=axes[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[1].plot(tr2.times(), data2, label=f"{tr2.stats.station} {tr2.stats.channel}", color=colors['tr2'], lw=1.0)
    axes[1].plot(tr2.times(), envelope2, color=colors['envelope2'], label='Envelope ' + tr2.stats.channel)
    axes[1].set_ylabel(f'v ({unit})', fontsize=font_ax)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)", fontsize=font_ax)  
    axes[1].legend(fontsize=font_leg)
    axes[1].text(0.02, 0.95, "b)", transform=axes[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')

    axes[2].plot(ff1, spec1, color=colors['spec1'], lw=1.0, label=f"Spectra {tr1.stats.station} {tr1.stats.channel}")
    axes[2].plot(ff2, spec2, color=colors['spec2'], lw=1.0, label=f"Spectra {tr2.stats.station} {tr2.stats.channel}")
    axes[2].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[4].set_xlabel(f"Frequenz (Hz)",fontsize=font_ax) 
    axes[2].legend(fontsize=font_leg)
    axes[2].text(0.02, 0.95, "c)", transform=axes[2].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[3].plot(ff1, spec1_s, color=colors['spec1'], lw=1.0, label=f"Spectra (smoothed) {tr1.stats.station} {tr1.stats.channel}")
    axes[3].plot(ff2, spec2_s, color=colors['spec2'], lw=1.0, label=f"Spectra (smoothed) {tr2.stats.station} {tr2.stats.channel}")
    axes[3].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font_ax)
    axes[3].legend(fontsize=font_leg)
    axes[3].text(0.02, 0.95, "d)", transform=axes[3].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[4].plot(band_centers, mean_ratios, marker='o', linestyle='None', color='black',
                 label=f"⌀ Spectral ratio {tr1.stats.station}/{tr2.stats.station}")
    axes[4].set_ylabel(f"⌀ Spectral ratio", fontsize=font_ax)
    axes[4].grid(True, which='both', linestyle='None', alpha=0.5)
    axes[4].legend(fontsize=font_leg)
    axes[4].text(0.02, 0.95, "e)", transform=axes[4].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[2].set_xlim(config['fmin'],config['fmax'])
        axes[3].set_xlim(config['fmin'],config['fmax'])
        axes[4].set_xlim(config['fmin'],config['fmax'])
       
        
    fig.suptitle(f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC", fontsize=font+4, y=0.92)

    result_dict = {
            "origin_time": str(origin_time),
            "region": region,
            "magnitude": magnitude,
            "distance": distance_km,
            "depth": ev_depth,
            "lat" : ev_lat,
            "lon": ev_lon,
            "backazimuth" : backazimuth
            }
    if not isinstance(ratio, int):
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.4f}Hz"] = mean_ratios[i]
    else:
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.3f}Hz"] = mean_ratio

    del st_in
    return fig, result_dict

In [14]:
#nur für translation --> nicht für rotation
def __makeplotspectraZsmooth(st, config, ev_lat, ev_lon, backazimuth):
    st_in = st.select(component="Z")
    
    if len(st_in) == 0:
        print(" -> keine Z-Komponente gefunden.")
        return

    trans_scaling = 1e6
    trans_unit = r"$\mathrm{\mu m/s}$"
    font = 14

    # Initialisierung
    tr1, tr2 = None, None
    
    # Stationen gezielt zuordnen
    for tr in st_in:
        if tr.stats.station.upper() == "FUR":
            tr1 = tr
            data1 = tr1.data * trans_scaling
            spec1, ff1, ph1 = __fft(data1 ,tr.stats.delta, window=None, normalize=None)
        elif tr.stats.station.upper() == "WET":
            tr2 = tr
            data2 = tr2.data * trans_scaling
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
    
    # Sicherstellen, dass beide Stationen gefunden wurden
    if tr1 is None or tr2 is None:
        print(" -> Fehlende Station: FUR oder WET nicht im Stream.")
        return
    if ev_lat > 30 and ev_lat < 60 and ev_lon > -50 and ev_lon < 50:
        region = 1
        print("Region 1 (Nahbeben)")
    elif ev_lat > 15 and ev_lat < 60 and ev_lon > 60 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > -10 and ev_lat < 15 and ev_lon > 100 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > 5 and ev_lat < 60 and ev_lon > -170 and ev_lon < -60:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -80 and ev_lat < 5 and ev_lon > -120 and ev_lon < -50:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -60 and ev_lat < -10 and (ev_lon > 90 or ev_lon < -150):
        region = 6
        print("Region 6 (Ozeanien)")
    else:
        region = 7
        print("Region 7 (nicht zugeordnet)")
    min_len = min(len(spec1), len(spec2), len(ff1), len(ff2))
    spec1 = spec1[:min_len]
    spec2 = spec2[:min_len]
    ff1 = ff1[:min_len]
    ff2 = ff2[:min_len]
    envelope1 = np.abs(hilbert(data1))
    envelope2 = np.abs(hilbert(data2))
    spec1_s = pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec2_s = pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec1_s = savgol_filter(spec1_s, window_length=81, polyorder=3)
    spec2_s = savgol_filter(spec2_s, window_length=81, polyorder=3)
    lines = ["⌀ Spectral ratio:"]
   
    ratio = safe_divide(spec1_s, spec2_s)   

    freq_bands = [(round(f, 5), round(f + 0.1, 5)) for f in np.arange(0.01, 1.9 + 0.001, 0.05)]
    # Mittelwerte und Mittenfrequenzen extrahieren
    band_centers = [(fmin + fmax)/2 for fmin, fmax in freq_bands]
    mean_ratios = []
    if not isinstance(ratio, int):
        for fmin_r, fmax_r in freq_bands:
            mask = (ff1 >= fmin_r) & (ff1 <= fmax_r)
            if np.any(mask):
             
                mean_ratios.append(np.mean(ratio[mask]))
            else:
                mean_ratios.append(np.nan)

     # Farben definieren
    colors = {
        'tr1': '#1f77b4',        # Blau
        'envelope1': '#2ca02c',  # Grün
        'tr2': '#ff7f0e',        # Orange
        'envelope2': '#17becf',  # Türkis
        'spec1': '#1f77b4',      # Blau
        'spec2': '#ff7f0e',      # Orange
    }   
    unit = trans_unit
    fig, axes = plt.subplots(5,figsize=(18,5*4), sharex=False)
    plt.subplots_adjust(hspace=0.3)

    axes[0].plot(tr1.times(), data1, label=f"{tr1.stats.station} {tr1.stats.channel}", color=colors['tr1'], lw=1.0)
    axes[0].plot(tr1.times(), envelope1, color=colors['envelope1'], label='Envelope ' + tr1.stats.channel) 
    axes[0].set_ylabel(f'envelope and v ({unit})', fontsize=font)
    axes[0].legend()
    axes[0].text(0.02, 0.95, "a)", transform=axes[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[1].plot(tr2.times(), data2, label=f"{tr2.stats.station} {tr2.stats.channel}", color=colors['tr2'], lw=1.0)
    axes[1].plot(tr2.times(), envelope2, color=colors['envelope2'], label='Envelope ' + tr2.stats.channel)
    axes[1].set_ylabel(f'Envelope and v ({unit})', fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)", fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)",fontsize=font)  
    axes[1].legend()
    axes[1].text(0.02, 0.95, "b)", transform=axes[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[2].plot(ff1, spec1, color=colors['spec1'], lw=1.0, label=f"Spectra {tr1.stats.station} {tr1.stats.channel}")
    axes[2].plot(ff2, spec2, color=colors['spec2'], lw=1.0, label=f"Spectra {tr2.stats.station} {tr2.stats.channel}")
    axes[2].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[2].set_xlabel(f"Frequency (Hz)",fontsize=font) 
    axes[2].legend()
    axes[2].text(0.02, 0.95, "c)", transform=axes[2].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[3].plot(ff1, spec1_s, color=colors['spec1'], lw=1.0, label=f"Spectra (smoothed) {tr1.stats.station} {tr1.stats.channel}")
    axes[3].plot(ff2, spec2_s, color=colors['spec2'], lw=1.0, label=f"Spectra (smoothed) {tr2.stats.station} {tr2.stats.channel}")
    axes[3].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[3].legend()
    axes[3].text(0.02, 0.95, "d)", transform=axes[3].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[4].plot(band_centers, mean_ratios, marker='o', linestyle='None', color='black',
                 label=f"⌀ Spectral ratio {tr1.stats.station}/{tr2.stats.station}")
    axes[4].set_ylabel(f"⌀ Spectral ratio", fontsize=font)
    axes[4].grid(True, which='both', linestyle='None', alpha=0.5)
    axes[4].legend()
    axes[4].text(0.02, 0.95, "e)", transform=axes[4].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[2].set_xlim(config['fmin'],config['fmax'])
        axes[3].set_xlim(config['fmin'],config['fmax'])
        axes[4].set_xlim(config['fmin'],config['fmax'])
       
        
    fig.suptitle(f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC", fontsize=font+2)

    result_dict = {
            "origin_time": str(origin_time),
            "region": region,
            "magnitude": magnitude,
            "distance": distance_km,
            "depth": ev_depth,
            "lat" : ev_lat,
            "lon": ev_lon,
            "backazimuth": backazimuth
            }
    if not isinstance(ratio, int):
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.4f}Hz"] = mean_ratios[i]
    else:
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.3f}Hz"] = mean_ratio

    del st_in
    return fig, result_dict

### Configurations

In [15]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# duration of event in seconds
config['duration'] = 7200

# frequency range for bandpass filter
config['fmin'] = 0.01 # in Hz   urspr. 0.01
config['fmax'] = 2 # in Hz    urspr. 0.1

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/Geschwindigkeit/FURWET/"

# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsFURWET/"

# specify seed codes of stations that should be used for the analysis
config['seeds'] = [#"BW.ROMY.10.BJZ", "BW.ROMY..BJU", "BW.ROMY..BJV", "BW.ROMY..BJW", # ringlaser ROMY
                   #"BW.RLAS..BJZ", # ringlaser G
                   "GR.FUR..BHZ", "GR.FUR..BHN", "GR.FUR..BHE", # seismometer ROMY
                   "GR.WET..BHZ", "GR.WET..BHN", "GR.WET..BHE", # seismometer G
                  ]

# path to catalogs
config['path_to_catalog'] = "C:/Bachelorarbeit/data/catalogs/"

# catalog file
config['catalog'] = "ROMY_global_catalog_20200101_20250531.pkl"

# set if figures should be saved
config['save_figures'] = True

# set if waveform data should be stored
config['store_waveforms'] = True

## Load Events

In [16]:
events = pd.read_pickle(config['path_to_catalog']+config['catalog'])

events['origin'] = events.timestamp

# make sure only events with magnitude > 6 are considered
events = events[events.magnitude > 6]
print("Event number: ", events.shape[0])

# avoid events that are too close to each other in time
events['elapsed_time'] = events.timestamp.diff()
events = events[events.elapsed_time > pd.Timedelta(minutes=60)]
print("Event number: ", events.shape[0])

output_path = config['outpath_figs'] + 'Katalog.csv'

# Speichert den DataFrame als CSV-Datei
events.to_csv(output_path, index=False)

Event number:  466
Event number:  385


# RUN LOOP

Loop over events in catalog and request data for each event. Store data or event plots if
- skip existing files
- save figures
- store waveforms

is set in configurations, respectively.



In [17]:
errors = []
results = []
results_T = []
results_Z = []
model = TauPyModel(model="prem")
fmin = config['fmin']
fmax = config['fmax']
filtered_path = config['outpath_figs'] + f"filtered_{fmin}_{fmax}/"
if not os.path.isdir(filtered_path):
            print("created: ", filtered_path)
            os.makedirs(filtered_path)
skip_indices = {3,4,1,2,5,6,9,10,14,20,24,28,33,36,37,40,41,44,45,46,48,49,47,51,56,57,59,60,61,62,72,70,72,71,78,82,86,96,98,99,100,104,105,108,109,110,
                113,115,123,124,125,126,127,128,129,130,131,132,133,135,136,137,144,146,156,157,167,170,171,173,174,175,178,180,181,182,183,184,185,
                189,190,193,196,199,201,202,204,207,208,209,210,211,212,213,215,220,222,223,228,229,230,232,233,237,238,239,240,241,242,245,246,247,248,
                249,250,253,254,255,256,258,259,260,263,264,265,266,268,272,273,274,275,276,278,279,280,281,282,283,284,285,287,288,293,294,297,298,299,
                303,307,311,313,314,318,324,327,328,332,345,346,349,353,355,356,357,358,359,360,364,366,370,372,379,380,384,15,22,27,29,32,52,97,120,
               139,151,161,163,205,227,243,295,306,316,317,319,331,348,369,377,374,383,269,235,216,187,168,159,79,17,0}
                
i = 0
# loop over all events
for jj in range(events.shape[0]):
    print(f"i={i}")
    print(f"jj={i}")
    if i != jj:
        print(f"i ungleich jj: i={i}, jj={jj}")
        break
    num = str(jj).rjust(3, "0")
    print(f"\n -> {num} {events.origin.iloc[jj]}")   
    try:
        event_name = str(events.origin.iloc[jj]).replace("-", "").replace(":", "").replace(" ", "_").split(".")[0]
    except:
        print(f" -> {num}: error for {events.origin.iloc[jj]}")
        i += 1
        continue

    # event metadata
    ev_lat = events.latitude.iloc[jj]
    ev_lon = events.longitude.iloc[jj]
    sta_lat = config['ROMY_lat']
    sta_lon = config['ROMY_lon']
    distance_m, az, backazimuth = gps2dist_azimuth(ev_lat, ev_lon, sta_lat, sta_lon)
    distance_km = distance_m / 1000
    ev_depth = events.depth.iloc[jj] / 1000
    #origin_time = events.origin.iloc[jj]
    origin_time = obs.UTCDateTime(events.origin.iloc[jj])
    magnitude = events.magnitude.iloc[jj]
    distance_deg = locations2degrees(ev_lat, ev_lon, sta_lat, sta_lon)
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg)
    
    if len(arrivals) == 0:
        print(f"Keine P-Ankunft für Event {jj} (Tiefe {ev_depth} km, Distanz {distance_deg}°)")
        starttime = origin_time
    else:
        p_arrival_time = arrivals[0].time 
        starttime = origin_time + p_arrival_time - 20
    # set config
    config['title'] = f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC"
    config['tbeg'] = starttime
    #config['tbeg'] = obs.UTCDateTime(str(origin_time))
    if distance_km < 2000:
        config['duration'] = 1000
    elif distance_km < 5000 and distance_km > 2000:
        config['duration'] = 1500
    elif distance_km < 8000 and distance_km > 5000:
        config['duration'] = 2500
    elif distance_km < 10000 and distance_km > 8000:
        config['duration'] = 3000
    elif distance_km < 14000 and distance_km > 10000:
        config['duration'] = 4000
    elif distance_km < 15000 and distance_km > 14000:
        config['duration'] = 4500
    else:
        config['duration'] = 7200
    if i == 85:
        config['duration'] = 4000
    if i == 111:
        config['duration'] = 2500
    if i == 217:
        config['duration'] = 3000
    if i == 226:
        config['duration'] = 1500
    if i == 244 or i == 300:
        config['tbeg'] = starttime-100
    i += 1
    config['tend'] = config['tbeg'] + config['duration']

    #check if data is aleready there
    if os.path.isfile(config['outpath_data']+f"{num}_{event_name}.mseed"):
        st0 = read(config['outpath_data']+f"{num}_{event_name}.mseed")
        data_vorhanden = True
        print(data_vorhanden)
    else:
        data_vorhanden = False
        st0 = obs.Stream()
        print(data_vorhanden)
    
        for seed in config['seeds']:
            try:
                if "FUR" in seed or "WET" in seed:
                    stx, invx = __querrySeismoData(
                        seed_id=seed,
                        starttime=config['tbeg'],
                        endtime=config['tend'],
                        repository='online',
                        path=None,
                        restitute=True,
                        detail=None,
                        fill_value=None,
                        )
                else:
                    print(f" -> {seed} not found")
                    continue
    
                if len(stx) == 0:
                    print(f" -> data missing for {seed}")
                else:
                    i=i
                   # event_status.loc[num, seed] = True
                st0 += stx

            except Exception as e:
                print(e)
                print(f" -> failed to request {seed} for event: {origin_time}")
                errors.append(f" -> failed to request {seed} for event: {origin_time}")
                continue
    if len(st0) < 6:
        continue
    # sort stream
    st0 = st0.sort()
    st0 = st0.select(station="FUR") + st0.select(station="WET")
    # fill masked data
    for tr in st0:
        if isinstance(tr.data, np.ma.MaskedArray):
            print(f" -> {tr.stats.channel} has masked data. Filled with zeros.")
            tr.data = tr.data.filled(fill_value=0)

    # preprocess
    print(" -> processing data stream ...")
    st1 = st0.copy()
    st1 = st1.detrend("linear")
    st1 = st1.taper(0.002)
    st1 = st1.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)
    print("gefiltert")
    # RTZ transformation pro Station
    st1_rtz = obs.Stream()
    for net_sta in set(f"{tr.stats.network}.{tr.stats.station}" for tr in st1):
        substream = st1.select(network=net_sta.split(".")[0], station=net_sta.split(".")[1])
        components = [tr.stats.channel[-1] for tr in substream]
        if all(c in components for c in ['N', 'E', 'Z']):
            try:
                substream_rtz = transform_to_rtz(substream, azimuth=az)
                st1_rtz += substream_rtz
                print(f" -> transformed {net_sta} to RTZ")
            except Exception as e:
                print(f" -> RTZ failed for {net_sta}: {e}")
                errors.append(f"RTZ failed for {net_sta} in event {num}")
                continue
                st1_rtz += substream
              
        else:
            print(f" -> skipping {net_sta}, missing components: {components}")
            st1_rtz += substream

    # trim streams
    st1_rtz = st1_rtz.trim(config['tbeg'], config['tend'])
    st0 = st0.trim(config['tbeg'], config['tend'])
    if len(st1_rtz) <5:
        continue
    # store waveforms
    if not data_vorhanden:
        waveform_filename = f"{num}_{event_name}.mseed"
        if not os.path.isdir(config['outpath_data']):
            print("created: ", config['outpath_data'])
            os.makedirs(config['outpath_data'])

        try:
            st0.write(config['outpath_data'] + waveform_filename)
            print(f" -> stored at: {config['outpath_data'] + waveform_filename}")
        except Exception as e:
            print(f" -> error storing waveform: {e}")
            errors.append(f" -> error storing waveform: {e}")
 
    # -- Speichern für T-Komponente
    fig8, results_t = __makeplotspectraTsmooth(st1_rtz, config, ev_lat, ev_lon, backazimuth)
    
    # Speichern als Einzeldatei pro Event
    dt = pd.DataFrame([results_t])
    t_result_path = config['outpath_figs'] + f"T_results/{num}_{event_name}.csv"
    dt.to_csv(t_result_path, index=False)
    
    fig8.savefig(filtered_path + f"{num}_{event_name}_Spectra_envelope_T_smooth.png", dpi=150, bbox_inches='tight', pad_inches=0.05)
    plt.close(fig8)
    fig8.clf()
    del fig8, dt, results_t
    gc.collect()
    
    # -- Speichern für Z-Komponente
    fig7, results_z = __makeplotspectraZsmooth(st1_rtz, config, ev_lat, ev_lon, backazimuth)
    
    df = pd.DataFrame([results_z])
    z_result_path = config['outpath_figs'] + f"Z_results/{num}_{event_name}.csv"
    df.to_csv(z_result_path, index=False)
    
    fig7.savefig(filtered_path + f"{num}_{event_name}_Spectra_envelope_Z-smooth.png", dpi=150, bbox_inches='tight', pad_inches=0.05)
    fig7.clf()
    plt.close(fig7)
    del fig7, df, results_z, arrivals
    st0 = None
    st1 = None
    st1_rtz = None
    gc.collect()

# show errors
pprint(errors)

# T-Komponente
t_files = glob.glob(config['outpath_figs'] + "T_results/*.csv")
df_all_T = pd.concat([pd.read_csv(f) for f in t_files], ignore_index=True)
df_all_T.to_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_T.csv", index=False)

# Z-Komponente
z_files = glob.glob(config['outpath_figs'] + "Z_results/*.csv")
df_all_Z = pd.concat([pd.read_csv(f) for f in z_files], ignore_index=True)
df_all_Z.to_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_Z.csv", index=False)

i=0
jj=0

 -> 000 2020-01-07 08:24:25.120000
skipped: 0
i=1
jj=1

 -> 001 2020-01-19 16:58:20.270000
skipped: 1
i=2
jj=2

 -> 002 2020-01-20 06:51:38.360000
skipped: 2
i=3
jj=3

 -> 003 2020-01-24 17:55:15.540000
skipped: 3
i=4
jj=4

 -> 004 2020-01-28 19:10:23.760000
skipped: 4
i=5
jj=5

 -> 005 2020-02-05 18:12:37.340000
skipped: 5
i=6
jj=6

 -> 006 2020-02-09 06:04:30.310000
skipped: 6
i=7
jj=7

 -> 007 2020-02-13 10:33:44.930000
skipped: 7
i=8
jj=8

 -> 008 2020-03-14 10:01:16.900000
skipped: 8
i=9
jj=9

 -> 009 2020-03-18 03:13:45.230000
skipped: 9
i=10
jj=10

 -> 010 2020-03-18 17:45:39.390000
skipped: 10
i=11
jj=11

 -> 011 2020-03-25 02:49:19.890000
skipped: 11
i=12
jj=12

 -> 012 2020-04-18 08:25:37.640000
skipped: 12
i=13
jj=13

 -> 013 2020-05-02 12:51:05.870000
skipped: 13
i=14
jj=14

 -> 014 2020-05-06 13:53:56.330000
skipped: 14
i=15
jj=15

 -> 015 2020-05-12 22:41:13.470000
skipped: 15
i=16
jj=16

 -> 016 2020-06-03 07:35:35.630000
skipped: 16
i=17
jj=17

 -> 017 2020-06

C:\Users\bened\AppData\Local\Temp\ipykernel_23180\695636560.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  spec1_s = pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
C:\Users\bened\AppData\Local\Temp\ipykernel_23180\695636560.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  spec2_s = pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')


Region 1 (Nahbeben)
i=44
jj=44

 -> 044 2020-11-07 01:10:30.790000
skipped: 44
i=45
jj=45

 -> 045 2020-11-22 00:54:55.240000
skipped: 45
i=46
jj=46

 -> 046 2020-11-29 16:40:43.660000
skipped: 46
i=47
jj=47

 -> 047 2020-11-30 22:54:35
skipped: 47
i=48
jj=48

 -> 048 2020-12-06 16:47:42.730000
skipped: 48
i=49
jj=49

 -> 049 2020-12-14 15:20:49.330000
skipped: 49
i=50
jj=50

 -> 050 2020-12-20 17:23:23.130000
skipped: 50
i=51
jj=51

 -> 051 2020-12-27 21:39:15.340000
skipped: 51
i=52
jj=52

 -> 052 2021-01-08 05:01:04.070000
skipped: 52
i=53
jj=53

 -> 053 2021-01-11 21:32:59.410000
skipped: 53
i=54
jj=54

 -> 054 2021-01-19 02:46:21.200000
skipped: 54
i=55
jj=55

 -> 055 2021-01-21 12:23:05.760000
skipped: 55
i=56
jj=56

 -> 056 2021-01-23 23:36:51.880000
skipped: 56
i=57
jj=57

 -> 057 2021-01-31 19:05:13.040000
skipped: 57
i=58
jj=58

 -> 058 2021-02-10 13:19:57.990000
skipped: 58
i=59
jj=59

 -> 059 2021-02-10 21:24:01.030000
skipped: 59
i=60
jj=60

 -> 060 2021-02-12 17:01:34.120